In [1]:
from Project.Database import Db
import pandas as pd
import plotly.express as px
from Project._07OptimiseConsumption.emission_reduction import emission_reduction
from Project._07OptimiseConsumption.optimisation_problem import power_consumption

In [2]:
NZERTF_optimisation_year1, NZERTF_emission_year1 = emission_reduction(year=1)

In [3]:
NZERTF_optimisation_year2, NZERTF_emission_year2 = emission_reduction(year=2)

In [4]:
NZERTF_optimisation_year2

{'w.r':                Timestamp  Day  Hour  Load_LatentHeatWaterVolume  \
 0    2015-02-01 00:00:00    0     0                    0.027909   
 1    2015-02-01 01:00:00    0     1                    0.095475   
 2    2015-02-01 02:00:00    0     2                    0.163375   
 3    2015-02-01 03:00:00    0     3                    0.227850   
 4    2015-02-01 04:00:00    0     4                    0.291952   
 ...                  ...  ...   ...                         ...   
 8755 2016-01-31 19:00:00  364    19                    1.434078   
 8756 2016-01-31 20:00:00  364    20                    1.509165   
 8757 2016-01-31 21:00:00  364    21                    1.582327   
 8758 2016-01-31 22:00:00  364    22                    1.654222   
 8759 2016-01-31 23:00:00  364    23                    1.726894   
 
       Load_RefrigeratorTemp  Load_StatusBA1Lights  Load_StatusKitchenLightsA  \
 0                  4.957915                   0.0                   0.000000   
 1           

In [5]:
for key, value in NZERTF_optimisation_year2.items():
    print(f'{key}: {value["Emission"]}')

w.r: 0       0.508536
1       0.727316
2       0.525879
3       0.508214
4       0.520379
          ...   
8755    1.642831
8756    1.360740
8757    1.407198
8758    0.939420
8759    0.602089
Name: Emission, Length: 8760, dtype: float64
w.o.r: 0       0.508536
1       0.727316
2       0.525879
3       0.508214
4       0.520379
          ...   
8755    1.462738
8756    1.278009
8757    1.407198
8758    0.926533
8759    0.600439
Name: Emission, Length: 8760, dtype: float64
m.a.o: 0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
8755    0.000000
8756    0.000000
8757    0.033034
8758    0.117626
8759    0.117626
Name: Emission, Length: 8760, dtype: float64
m.a.u.o: 0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
8755    0.000000
8756    0.108122
8757    0.153689
8758    0.079059
8759    0.000000
Name: Emission, Length: 8760, dtype: float64


In [50]:
NZERTF_emission_year1

{'w.r': 7067.381688897962,
 'w.o.r': 6930.435837229237,
 'm.a.o': 670.645964402705,
 'm.a.u.o': 680.062579708695}

In [6]:
NZERTF_emission_year2

{'w.r': 8388.657846296948,
 'w.o.r': 8257.25142366896,
 'm.a.o': 533.8225279357321,
 'm.a.u.o': 528.1785737581545}

In [51]:
redundancy_reduction = NZERTF_emission_year1['w.r'] - NZERTF_emission_year1['w.o.r']
optimisation_reduction = NZERTF_emission_year1['m.a.u.o'] - NZERTF_emission_year1['m.a.o']
total_reduction = redundancy_reduction + optimisation_reduction
redundancy_reduction, optimisation_reduction, total_reduction

(136.94585166872457, 9.416615305990035, 146.3624669747146)

In [7]:
redundancy_reduction = NZERTF_emission_year2['w.r'] - NZERTF_emission_year2['w.o.r']
optimisation_reduction = NZERTF_emission_year2['m.a.u.o'] - NZERTF_emission_year2['m.a.o']
total_reduction = redundancy_reduction + optimisation_reduction

In [49]:
redundancy_reduction, optimisation_reduction, total_reduction

(131.40642262798792, -5.643954177577598, 125.76246845041032)

In [11]:
NZERTF_optimisation_year2['m.a.o']

,Timestamp,Day,Hour,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission(g/Wh),Emission,Load_StatusApplianceDishwasherEmission,Load_StatusPlugLoadVacuumEmission,Load_StatusClothesWasherEmission,Load_StatusDryerPowerTotalEmission,Load_StatusPlugLoadIronEmission
0,2015-02-01 00:00:00,0,0,0.0000,0.0,0.0,0.0,0.0,0.734305,0.000000,0.000000,0.0,0.0,0.0,0.0
1,2015-02-01 01:00:00,0,1,0.0000,0.0,0.0,0.0,0.0,0.752407,0.000000,0.000000,0.0,0.0,0.0,0.0
2,2015-02-01 02:00:00,0,2,0.0000,0.0,0.0,0.0,0.0,0.737095,0.000000,0.000000,0.0,0.0,0.0,0.0
3,2015-02-01 03:00:00,0,3,0.0000,0.0,0.0,0.0,0.0,0.733298,0.000000,0.000000,0.0,0.0,0.0,0.0
4,2015-02-01 04:00:00,0,4,0.0000,0.0,0.0,0.0,0.0,0.731043,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2016-01-31 19:00:00,364,19,0.0000,0.0,0.0,0.0,0.0,0.684510,0.000000,0.000000,0.0,0.0,0.0,0.0
8756,2016-01-31 20:00:00,364,20,0.0000,0.0,0.0,0.0,0.0,0.697113,0.000000,0.000000,0.0,0.0,0.0,0.0
8757,2016-01-31 21:00:00,364,21,0.2167,0.0,0.0,0.0,0.0,0.544998,0.033034,0.033034,0.0,0.0,0.0,0.0
8758,2016-01-31 22:00:00,364,22,1.0000,0.0,0.0,0.0,0.0,0.420529,0.117626,0.117626,0.0,0.0,0.0,0.0


In [12]:
NZERTF_optimisation_year2['w.r']['Consumption'].sum()-NZERTF_optimisation_year2['w.o.r']['Consumption'].sum()

182974.74609446712

In [13]:
NZERTF_optimisation_year2['w.r']['Consumption'].groupby(NZERTF_optimisation_year2['w.r']['Consumption'].index % 24).mean()

0      698.788280
1      709.881802
2      702.950804
3      725.324320
4      772.492298
5      739.047235
6     2977.813052
7     1597.746481
8     1640.184101
9     1367.349349
10    1109.436437
11     559.195413
12    1260.300588
13     710.009025
14    1027.502203
15    1109.782531
16     665.280911
17     748.913165
18    2941.576315
19    2525.628646
20    2473.697660
21    2272.299162
22    1995.166929
23    1131.257300
Name: Consumption, dtype: float64

## Emission reduction from redundancy removal

In [14]:
meta = Db.load_data(year=2, hourly=False, consumption=False, production=False, meta=True, with_redundancy=True)

In [15]:
status_constraint = meta.loc[(lambda self:
                           (~self['Consumer_Match'].isna()) &
                           (~self.index.str.contains("SensHeat")) &
                           (self["Subsystem"] == "Loads"))].index.to_list()

In [53]:
appliance_emission_diff_year1 = NZERTF_optimisation_year1['w.r'].copy()[[status + 'Emission' for status in status_constraint]].sum() - NZERTF_optimisation_year1['w.o.r'].copy()[[status + 'Emission' for status in status_constraint]].sum()

In [54]:
appliance_emission_diff_year2 = NZERTF_optimisation_year2['w.r'].copy()[[status + 'Emission' for status in status_constraint]].sum() - NZERTF_optimisation_year2['w.o.r'].copy()[[status + 'Emission' for status in status_constraint]].sum()

In [55]:
appliance_emission_diff = pd.concat([appliance_emission_diff_year1, appliance_emission_diff_year2], axis=1)
appliance_emission_diff.rename(columns={0: 'year1', 1: 'year2'},
                           index={index: index.split('Status')[-1].split('Emission')[0] for index in appliance_emission_diff.index},
                           inplace=True)
appliance_emission_diff = appliance_emission_diff.loc[lambda self: self.sum(1) > 0].sort_values(by='year1', ascending=False).multiply(1000)

In [58]:
appliance_emission_diff.round(2)

,year1,year2
PlugLoadBR2Laptop,9158.19,9180.48
PlugLoadIron,8622.50,8060.28
PlugLoadDesktopPCMonitor,6923.15,6904.55
BR2Lights,5798.11,5592.95
PlugLoadBR3Laptop,5720.85,5675.20
BR3Lights,4608.88,4593.68
PlugLoadCanOpener,714.56,0.00
PlugLoadLRTV,446.04,438.69
LRLights2,321.65,0.00
BA2Lights,212.19,211.98


In [56]:
fig = px.bar(appliance_emission_diff, barmode="group", text_auto='.2s', log_y=True,
             labels={'value': 'Redundant emissions (g)', 'index': 'Appliances'})
fig.update_layout(template='plotly')
fig.write_html(Db.get_save_file_directory(f"Bar/redundant_emission_per_appliance.html"))

## Hours and kWh reduced by redundancy removal

In [17]:
before_red_remov_year2 = NZERTF_optimisation_year2['w.r'][status_constraint].sum()
before_red_remov_year2.sum()

36004.79570102835

In [18]:
after_red_remov_year2 = NZERTF_optimisation_year2['w.o.r'][status_constraint].sum()
after_red_remov_year2.sum()

35001.24298024334

In [19]:
appliance_hour_diff_year2 = before_red_remov_year2-after_red_remov_year2

In [20]:
(before_red_remov_year2.sum()-after_red_remov_year2.sum())/before_red_remov_year2.sum()*100

2.787275142784235

In [21]:
before_red_remov_year1 = NZERTF_optimisation_year1['w.r'][status_constraint].sum()
before_red_remov_year1.sum()
after_red_remov_year1 = NZERTF_optimisation_year1['w.o.r'][status_constraint].sum()
after_red_remov_year1.sum()
appliance_hour_diff_year1 = before_red_remov_year1 - after_red_remov_year1

In [45]:
appliance_hour_diff = pd.concat([appliance_hour_diff_year1, appliance_hour_diff_year2], axis=1)
appliance_hour_diff.rename(columns={0: 'year1', 1: 'year2'},
                      index={index: index.split('Status')[-1] for index in appliance_hour_diff.index}, inplace=True)
appliance_hour_diff = appliance_hour_diff.loc[lambda self: self.sum(1) > 0].sort_values(by='year1', ascending=False)

In [46]:
import plotly.express as px

fig = px.bar(appliance_hour_diff, barmode="group",text_auto='.2s', log_y=True, labels={'value':'Redundant hours', 'index': 'Appliances'})
fig.update_layout(template='plotly')
fig.write_html(Db.get_save_file_directory(f"Bar/redundant_hours_per_appliance.html"))

## kWh

In [24]:
power_consumption = power_consumption(status_constraint)

In [25]:
before_red_remov_kWh = NZERTF_optimisation_year2['w.r'].copy()[status_constraint].multiply(power_consumption).sum()

In [26]:
after_red_remov_kWh = NZERTF_optimisation_year2['w.o.r'].copy()[status_constraint].multiply(power_consumption).sum()

In [27]:
appliance_diff_year2 = before_red_remov_kWh-after_red_remov_kWh

In [28]:
appliance_diff_year2/before_red_remov_kWh*100

Load_StatusBA1Lights                     0.000000
Load_StatusKitchenLightsA                0.000000
Load_StatusKitchenLightsB                0.000000
Load_StatusKitchenLightsC                0.000000
Load_StatusDRLights                      0.000000
Load_StatusLRLights3                     0.000000
Load_StatusEntryHallLights                    NaN
Load_StatusBR4Lights                          NaN
Load_StatusMBALights                     0.000000
Load_StatusMBRLights1                    0.002423
Load_StatusMBRLights2                    0.002423
Load_StatusBA2Lights                     6.532195
Load_StatusBR2Lights                    13.390014
Load_StatusBR3Lights                    26.593327
Load_StatusLRLights1                          NaN
Load_StatusLRLights2                     0.000000
Load_StatusPlugLoadBlender               0.000000
Load_StatusPlugLoadToasterOven           0.000000
Load_StatusApplianceRangeHood            0.000000
Load_StatusApplianceCooktop              0.000000


In [29]:
before_red_remov = NZERTF_optimisation_year1['w.r'].copy()[status_constraint].multiply(power_consumption).sum()
after_red_remov = NZERTF_optimisation_year1['w.o.r'].copy()[status_constraint].multiply(power_consumption).sum()
appliance_diff_year1 = before_red_remov - after_red_remov

In [30]:
appliance_diff = pd.concat([appliance_diff_year1,appliance_diff_year2], axis=1)
appliance_diff.rename(columns={0: 'year1', 1: 'year2'}, index= {index: index.split('Status')[-1] for index in appliance_diff.index}, inplace=True)

In [31]:
appliance_diff = appliance_diff.loc[lambda self: self.sum(1) > 0].sort_values(by='year1', ascending=False)

In [42]:
fig = px.bar(appliance_diff, barmode="group",text_auto='.2s', log_y=True, labels={'value':'Redundant Watt hours', 'index': 'Appliances'})
fig.update_layout(template='plotly')
fig.write_html(Db.get_save_file_directory(f"Bar/redundant_energy_per_appliance.html"))

## Emission reduction from optimisation

In [60]:
movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum','Load_StatusClothesWasher', 'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

In [72]:
appliance_emission_optim_diff_year1 = NZERTF_optimisation_year1['m.a.u.o'].copy()[
                                    [status + 'Emission' for status in movable_appliances]].sum() - NZERTF_optimisation_year1['m.a.o'].copy()[
                                    [status + 'Emission' for status in movable_appliances]].sum()
appliance_emission_optim_diff_year2 = NZERTF_optimisation_year2['m.a.u.o'].copy()[
                                    [status + 'Emission' for status in movable_appliances]].sum() - NZERTF_optimisation_year2['m.a.o'].copy()[
                                    [status + 'Emission' for status in movable_appliances]].sum()
appliance_emission_optim_diff = pd.concat([appliance_emission_optim_diff_year1, appliance_emission_optim_diff_year2], axis=1)
appliance_emission_optim_diff.rename(columns={0: 'year1', 1: 'year2'},
                                     index={index: index.split('Status')[-1].split('Emission')[0] for index in appliance_emission_optim_diff.index}, inplace=True)
appliance_emission_optim_diff = appliance_emission_optim_diff.sort_values(by='year1', ascending=False).multiply(1000)

In [75]:
appliance_emission_optim_diff

,year1,year2
ApplianceDishwasher,5121.987980,7231.130484
PlugLoadIron,3260.740673,2763.167474
PlugLoadVacuum,800.766068,1199.690137
ClothesWasher,466.036049,2216.581320
DryerPowerTotal,-232.915464,-19054.523593


In [90]:
fig = px.bar(appliance_emission_optim_diff, barmode="group",text_auto='.2s', labels={'value':'Optimised CO2 emission (g) reduction', 'index': 'Appliances'}, )
fig.update_layout(template='plotly')
fig.write_html(Db.get_save_file_directory(f"Bar/optimised_emission_per_appliance.html"))